# 🎮 Manual Control - Return to Home

Simple remote control to manually drive the JetBot back to starting position.

**No camera needed** - just joystick control.

## Controls:
- **Left Stick Y**: Forward / Backward
- **Left Stick X**: Turn left / right
- **Right Trigger (RT)**: Speed boost
- **Left Trigger (LT)**: Slow mode (precision)

In [ ]:
# =============================================================================
# MANUAL CONTROL - No Camera
# =============================================================================

import time
import threading
import ipywidgets as widgets
from IPython.display import display
from jetbot import Robot

# -----------------------------------------------------------------------------
# CONFIGURATION
# -----------------------------------------------------------------------------
BASE_SPEED = 0.15       # Normal driving speed
SLOW_SPEED = 0.08       # Precision mode (LT held)
BOOST_SPEED = 0.25      # Fast mode (RT held)
TURN_GAIN = 0.6         # How much turning affects wheels (0-1)
DEADZONE = 0.1          # Joystick deadzone

# Joystick mapping (Logitech F710)
AXIS_FORWARD = 1        # Left stick Y (inverted: up = negative)
AXIS_TURN = 0           # Left stick X
LT_BUTTON = 6           # Left trigger (slow mode)
RT_BUTTON = 7           # Right trigger (speed boost)

# -----------------------------------------------------------------------------
# SETUP
# -----------------------------------------------------------------------------
robot = Robot()

# Widgets
speed_label = widgets.Label(value='Speed: Normal')
left_motor_bar = widgets.FloatProgress(value=0.5, min=0, max=1, description='Left:', bar_style='info')
right_motor_bar = widgets.FloatProgress(value=0.5, min=0, max=1, description='Right:', bar_style='info')
status_label = widgets.Label(value='Ready - Press START')
start_btn = widgets.Button(description='START', button_style='success', layout=widgets.Layout(width='150px', height='50px'))
stop_btn = widgets.Button(description='STOP', button_style='danger', layout=widgets.Layout(width='150px', height='50px'))
controller = widgets.Controller()

running = False

def apply_deadzone(value):
    if abs(value) < DEADZONE:
        return 0.0
    return value

def control_loop():
    global running
    
    while running:
        try:
            # Read joystick
            forward = -controller.axes[AXIS_FORWARD].value  # Invert Y axis
            turn = controller.axes[AXIS_TURN].value
            lt = controller.buttons[LT_BUTTON].value  # 0-1
            rt = controller.buttons[RT_BUTTON].value  # 0-1
        except:
            forward, turn, lt, rt = 0.0, 0.0, 0.0, 0.0
        
        # Apply deadzone
        forward = apply_deadzone(forward)
        turn = apply_deadzone(turn)
        
        # Determine speed mode
        if lt > 0.5:
            speed = SLOW_SPEED
            speed_label.value = '🐢 Speed: SLOW (precision)'
        elif rt > 0.5:
            speed = BOOST_SPEED
            speed_label.value = '🚀 Speed: FAST (boost)'
        else:
            speed = BASE_SPEED
            speed_label.value = '🚗 Speed: Normal'
        
        # Calculate motor values (arcade drive)
        left = forward * speed + turn * speed * TURN_GAIN
        right = forward * speed - turn * speed * TURN_GAIN
        
        # Clamp values
        left = max(-1, min(1, left))
        right = max(-1, min(1, right))
        
        # Drive
        robot.left_motor.value = left
        robot.right_motor.value = right
        
        # Update UI
        left_motor_bar.value = (left + 1) / 2  # Convert -1,1 to 0,1
        right_motor_bar.value = (right + 1) / 2
        
        if forward != 0 or turn != 0:
            status_label.value = f'Driving | L: {left:.2f} | R: {right:.2f}'
        else:
            status_label.value = 'Stopped - move joystick to drive'
        
        time.sleep(0.05)  # 20 Hz
    
    robot.stop()
    status_label.value = 'STOPPED'

def on_start(b):
    global running
    if not running:
        running = True
        status_label.value = 'Running - use joystick to drive'
        threading.Thread(target=control_loop, daemon=True).start()

def on_stop(b):
    global running
    running = False
    robot.stop()

start_btn.on_click(on_start)
stop_btn.on_click(on_stop)

# Display
print("🎮 Manual Control Ready")
print("━" * 40)
print("Left Stick  → Forward/Back + Turn")
print("LT (hold)   → Slow/Precision mode")
print("RT (hold)   → Fast/Boost mode")
print("━" * 40)

display(controller)
display(widgets.VBox([
    speed_label,
    left_motor_bar,
    right_motor_bar,
    status_label
]))
display(widgets.HBox([start_btn, stop_btn]))

In [ ]:
# Emergency stop / cleanup
running = False
robot.stop()
print("🛑 Robot stopped.")